In [ ]:
import numpy as np
import pandas as pd
import os
import sys
sys.path.append("..")

import io

from pathlib import Path
from tqdm.notebook import tqdm
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')# You must grant COLAB access to your Google Drive

Mounted at /content/drive


## Агрегация предсказаний от разных моделей



In [ ]:
rn = pd.read_csv("/content/subm_11_ss_rn50.csv")
rn_2 = pd.read_csv("/content/subm_17_ss_rn50.csv")
b3 = pd.read_csv("/content/subm_10_ssb3.csv")
b3_2 = pd.read_csv("/content/subm_08_b3.csv")
b5 = pd.read_csv("/content/subm_15_ss2b5.csv") 
wv = pd.read_csv("/content/wavenet_17.csv")
rn

,recording_id,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
0,000316da7,-9.079918,-10.607855,-12.940112,13.993675,-12.483793,-8.202158,-6.211764,-8.410800,-12.422962,-10.481474,-14.264782,-10.584187,5.066486,-13.103785,-9.278639,-9.568863,-11.413799,-11.658859,3.612096,-12.150081,-12.419035,-12.840101,-12.674565,-9.185518
1,003bc2cb2,-8.104124,-7.152707,-9.761471,18.148243,-11.913236,-7.111510,-7.827536,-4.331820,-10.312015,-10.075088,-9.352195,-7.919694,-5.319877,-8.902734,-6.512736,-7.612498,18.874468,-6.451684,-9.307724,-10.547099,-9.656973,-9.665983,-5.542996,-7.149926
2,0061c037e,-0.208325,-12.178954,-18.824548,-3.218816,-17.272503,6.343920,-15.529011,-7.666697,-18.929325,-16.830319,-13.349484,-11.748473,-14.634572,-20.421748,-16.704092,-14.022940,-16.926897,-17.075036,-16.971864,-16.322641,-11.930072,-19.102990,-9.015648,-11.988482
3,010eb14d3,17.620938,-11.499112,-14.223594,-8.382106,-13.766003,-9.277802,-14.215189,-10.046807,9.805230,-12.602936,-13.555869,-11.624607,-11.645260,-14.677137,-12.385055,-11.030452,-13.654458,-13.223007,-6.397167,-16.348594,-14.624572,-13.969676,-15.450043,-12.752290
4,011318064,-12.418292,-12.158210,-13.322700,0.405587,-11.455017,-10.454402,-10.926766,-10.068595,-15.055234,-14.882384,-12.176836,-11.560790,-11.628995,-13.929198,16.731450,1.035982,-12.573505,-13.300893,2.936429,-13.402689,-13.600714,-11.732993,-13.117738,-10.937462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987,ff68f3ac3,-13.013053,-12.709626,-12.961713,-3.529828,-5.644276,12.316151,-12.361350,-10.558715,-14.655917,-12.981415,-13.558468,-12.505374,-9.195741,-10.717352,-11.841939,3.841049,-12.541893,-12.396856,-12.045477,-14.339219,-5.974619,-14.376211,-13.142960,18.147969
1988,ff973e852,-12.175160,-13.777680,-16.769767,-9.701577,-17.894155,-1.335279,-11.555019,11.081891,-15.613228,-10.821954,-13.698923,-13.436974,-12.056955,-18.644141,-10.110063,2.786782,-15.145140,-12.291760,-13.234514,-16.269652,-5.180106,-17.704448,-12.641667,-9.727612
1989,ffa5cf6d6,-7.048441,-8.101792,-9.439806,17.270914,-11.489488,-3.896298,-7.591247,-4.285978,-11.229406,-6.901796,-9.255266,-8.692625,-5.643369,-4.315195,-7.151018,21.010717,-8.720651,-8.188825,-8.455714,-10.752682,-9.252051,-10.583470,-6.830079,-5.812594
1990,ffa88cbb8,-7.344902,7.265962,-9.412866,19.134843,-11.218938,-5.955967,-10.583252,-3.993025,-11.243254,-11.568196,-9.122059,-9.021183,-5.894478,-10.346839,-7.165396,-7.897678,-8.083047,-8.660250,-8.180025,-11.269171,-10.119113,-8.239315,-8.368892,-6.259373


In [ ]:
rn_ar = rn[rn.columns[1:]].to_numpy()
rn_fin = (rn_ar - rn_ar.flatten().mean()) / rn_ar.flatten().std()
rn_fin.max(axis=1)

array([3.12773814, 3.78385263, 2.09939806, ..., 4.07102405, 3.8188543 ,
       3.75005645])

In [ ]:
rn_2_ar = rn_2[rn_2.columns[1:]].to_numpy()
rn_2_fin = (rn_2_ar - rn_2_ar.flatten().mean()) / rn_2_ar.flatten().std()
rn_2_fin.max(axis=1)

array([3.64150516, 4.05140575, 2.10839486, ..., 4.16328515, 3.68096957,
       3.61372227])

In [ ]:
b3_ar = b3[b3.columns[1:]].to_numpy()
b3_fin = (b3_ar - b3_ar.flatten().mean()) / b3_ar.flatten().std()
b3_fin.max(axis=1)

array([2.52705097, 3.90168537, 1.96065628, ..., 3.05166689, 2.69067319,
       3.09325164])

In [ ]:
b3_2_ar = b3_2[b3_2.columns[1:]].to_numpy()
b3_2_fin = (b3_2_ar - b3_2_ar.flatten().mean()) / b3_2_ar.flatten().std()
b3_2_fin.max(axis=1)

array([2.91181807, 4.2033147 , 1.29174123, ..., 4.16676375, 3.75788344,
       3.38581759])

In [ ]:
b5_ar = b5[b5.columns[1:]].to_numpy()
b5_fin = (b5_ar - b5_ar.flatten().mean()) / b5_ar.flatten().std()
b5_fin.max(axis=1)

array([2.57598324, 4.23240892, 2.89009432, ..., 3.32010175, 1.95829439,
       3.08600479])

In [ ]:
wv_ar = wv[wv.columns[1:]].to_numpy()
wv_fin = (wv_ar - wv_ar.flatten().mean()) / wv_ar.flatten().std()
wv_fin.max(axis=1)

array([2.49413166, 4.52656041, 1.92286246, ..., 4.1826286 , 2.76780424,
       3.33879843])

In [ ]:
subm = pd.DataFrame(np.concatenate([rn[rn.columns[0]].to_numpy().reshape(-1,1), rn_fin + rn_2_fin + b3_fin + b3_2_fin + b5_fin + wv_fin ], axis=1), columns = rn.columns)
subm
subm["top_cat"] = subm[subm.columns[1:-1]].apply(lambda x: np.argsort(x)[::-1][:6].to_list(), axis=1)
subm[["recording_id" , "top_cat"]].head(10)

,recording_id,top_cat
0,000316da7,"[3, 12, 18, 0, 7, 14]"
1,003bc2cb2,"[16, 3, 12, 7, 1, 15]"
2,0061c037e,"[5, 0, 3, 7, 22, 11]"
3,010eb14d3,"[0, 8, 18, 3, 7, 5]"
4,011318064,"[14, 18, 15, 3, 7, 0]"
5,01307e693,"[7, 3, 15, 12, 17, 11]"
6,013bd8b85,"[12, 3, 18, 13, 9, 5]"
7,01812f522,"[17, 3, 16, 12, 11, 7]"
8,019db5220,"[5, 12, 22, 3, 18, 7]"
9,01bd97dd6,"[3, 12, 18, 7, 5, 14]"


In [ ]:
subm.drop(["top_cat"], axis=1, inplace=True)
subm.to_csv("subm_sum_17_top6_3.csv", index=False) # rn_fin + rn_2_fin + b3_fin + b3_2_fin + wv_fin + b5_fin

In [ ]:
subm.drop(["top_cat"], axis=1, inplace=True)
subm.to_csv("subm_sum_17_top5_2.csv", index=False) # rn_fin + rn_2_fin + b3_fin + b3_2_fin + wv_fin

In [ ]:
subm_pos = pd.read_csv("/content/subm1")
subm_pos["top_cat"] = subm_pos[subm_pos.columns[2:]].apply(lambda x: np.argsort(x)[::-1][:6].to_list(), axis=1)

In [ ]:
subm_pos[["recording_id" , "top_cat"]].head(10)

,recording_id,top_cat
0,000316da7,"[3, 9, 5, 18, 15, 12]"
1,003bc2cb2,"[16, 3, 7, 12, 13, 6]"
2,0061c037e,"[3, 7, 5, 17, 22, 20]"
3,010eb14d3,"[8, 0, 18, 3, 1, 9]"
4,011318064,"[14, 15, 18, 3, 9, 7]"
5,01307e693,"[7, 3, 15, 17, 12, 11]"
6,013bd8b85,"[12, 3, 13, 18, 5, 15]"
7,01812f522,"[17, 3, 16, 20, 12, 7]"
8,019db5220,"[23, 5, 22, 3, 18, 7]"
9,01bd97dd6,"[3, 12, 7, 23, 10, 15]"
